# Season 19 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [1]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
from blaseball_mike.tables import StatType
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown
from copy import deepcopy

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

# Fix Attractors
real_pies = deepcopy(pies)
for p in real_pies.lineup + real_pies.rotation:
    p.batting_rating = None
    p.pitching_rating = None
    p.baserunning_rating = None
    p.defense_rating = None

In [2]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 19, Day 79**

---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decrees will pass this season.

### Repeal Turntables
**Repeal the Turntables Rule.**

Repeals the Turntables Rule.

### Under Achiever

**unwin the underbracket. crown the underchampions.**


In [3]:
display(HTMLWrapper("<span style=\"font-family: 'Lora', 'Courier New', monospace, serif;font-weight: 700;color: #a16dc3;text-shadow:0 0 10px #a16dc3;\">as above, so below</span>"))

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Equivalent Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team. They must be within 2 combined stars of each other.**

See below for the 20 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [4]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)

In [5]:
all_total_stars = get_total_stars(all_wild)
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Eduardo Woodman

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Eduardo Woodman,Pies,5.5,2.0,3.5,3.5
Howell Franklin,Sunbeams,6.0,1.5,2.5,2.5
Dudl-y -u-ller,Spies,5.5,3.0,3.5,4.5
Margarito Nava,Flowers,5.5,3.5,4.5,2.5
Baby Doyle,Jazz Hands,5.0,2.5,3.0,4.5
Scores Baserunner,Flowers,5.0,2.5,4.0,4.5
Baby Triumphant,Firefighters,5.0,2.5,3.0,3.5
Math Velazquez,Spies,5.0,3.5,4.0,2.5
Comfort Septemberish,Spies,5.0,1.0,5.0,2.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Howell Franklin,1.226470,1.107821,1.353333,1.712056,1.274572,0.001000,1.217685,0.001000,1.402964
Dudl-y -u-ller,1.079778,1.644472,0.776783,0.987971,1.021591,0.182529,1.023215,0.001000,1.719870
Margarito Nava,1.072049,0.867560,1.315733,1.320216,1.456526,0.444288,1.197500,0.001000,0.858627
Baby Doyle,1.029355,1.254587,0.963011,1.303270,0.580199,0.311433,1.186896,0.101000,1.241413
Scores Baserunner,1.020751,1.255932,1.044977,0.719606,0.603396,0.001000,0.549101,0.001000,0.482450
Baby Triumphant,0.989831,1.017782,1.175039,0.680555,0.786115,0.001000,0.274271,0.001000,0.913858
Math Velazquez,0.981884,1.027667,1.067639,0.709586,1.146343,0.395484,0.622834,0.060000,0.682863
Comfort Septemberish,0.978574,1.177996,1.187902,0.375357,0.767285,0.611622,1.105479,0.060000,0.832023
Basilio Mason,0.956207,1.032192,0.794204,1.099347,1.510854,0.278276,1.163940,0.001000,1.498068


####Beasley Day

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Beasley Day,Pies,4.5,3.0,3.5,3.0
Howell Franklin,Sunbeams,6.0,1.5,2.5,2.5
Dudl-y -u-ller,Spies,5.5,3.0,3.5,4.5
Margarito Nava,Flowers,5.5,3.5,4.5,2.5
Baby Doyle,Jazz Hands,5.0,2.5,3.0,4.5
Scores Baserunner,Flowers,5.0,2.5,4.0,4.5
Baby Triumphant,Firefighters,5.0,2.5,3.0,3.5
Math Velazquez,Spies,5.0,3.5,4.0,2.5
Comfort Septemberish,Spies,5.0,1.0,5.0,2.5


...

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Peanut Holloway,Pies,6.0,4.0,4.0,5.5
Nagomi Nava,Sunbeams,6.0,3.0,5.0,3.5
Rivers Clembons,Dale,5.5,4.5,5.5,5.0
Richmond Harrison,Dale,5.0,3.0,5.5,6.0
Mcdowell Mason,Tacos,5.0,3.0,5.0,5.0
Peanutiel Duffy,Firefighters,4.5,5.0,3.0,4.0
Scratch Deleuze,Worms,4.5,2.5,5.5,4.0
Yurts Buttercup,Georgias,4.5,4.5,3.5,4.0
Loubert Ji-Eun,Worms,4.5,3.5,5.5,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Peanut Holloway,1.171131,0.656788,0.749014,0.650239,0.755549,0.234531,0.377807,0.207685,0.311857
Nagomi Nava,1.222305,0.936533,1.737007,0.860811,1.842572,0.030067,0.876832,0.001000,0.986954
Rivers Clembons,1.074366,0.931926,1.362447,0.571397,1.420467,0.001000,1.213424,0.001000,1.147909
Richmond Harrison,0.970436,0.913734,1.293920,0.568370,0.694178,0.122449,0.539844,0.001000,0.410669
Mcdowell Mason,0.967311,0.654408,1.346949,1.138806,0.915926,0.001000,1.479001,0.001000,0.807260
Peanutiel Duffy,0.932907,1.162867,0.795669,1.164276,0.767924,0.302006,0.454241,0.010311,0.389558
Scratch Deleuze,0.900960,1.125916,0.595994,1.205473,1.310680,0.001000,1.273637,0.313625,1.479767
Yurts Buttercup,0.895192,0.689838,1.277678,1.191714,0.563110,0.519699,1.324784,0.427045,1.213765
Loubert Ji-Eun,0.894272,1.174171,0.747799,1.266132,0.372224,0.001000,0.614355,0.001000,0.434761
Burke Go-zale-,0.885530,0.972925,0.718055,1.008684,1.271060,0.001000,0.481530,0.001000,1.252486


####Usurper Violet

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Usurper Violet,Pies,3.0,3.0,1.5,4.0
Howell Franklin,Sunbeams,6.0,1.5,2.5,2.5
Rat Mason,Tacos,5.0,1.5,2.5,1.5
Summers Presto-,Dale,5.0,1.5,1.5,2.0
Randy Castillo,Tigers,4.5,1.5,2.0,2.0
Reese Clark,Spies,4.5,3.0,2.0,4.0
Felix Garbage,Tacos,4.5,3.5,2.0,2.5
Fran Beans,Wild Wings,4.5,3.5,3.0,3.0
Denzel Scott,Spies,4.5,2.5,2.5,2.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Howell Franklin,1.226470,1.107821,1.353333,1.712056,1.274572,0.001000,1.217685,0.001000,1.402964
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Summers Presto-,0.974990,0.898723,1.244346,1.227772,0.360495,0.001000,0.848852,0.001000,0.783555
Randy Castillo,0.868039,1.030142,1.096949,0.686627,0.257003,0.629329,0.804416,0.022455,1.085623
Reese Clark,0.866074,0.897601,0.825922,0.776604,1.028210,0.358106,1.031754,0.060000,0.896331
Felix Garbage,0.859852,0.766287,1.174788,0.656731,0.329195,0.001000,1.036772,0.001000,0.680168
Fran Beans,0.855356,0.927986,1.053483,0.619330,0.552038,0.623110,0.386358,0.001000,0.933028
Denzel Scott,0.850411,0.963515,0.849616,0.566107,0.627399,0.119743,0.673793,0.007747,0.949707
Nicholas Mora,0.828365,1.058400,0.816198,0.246713,0.748028,0.100439,0.846379,0.157737,0.541933


In [6]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Henry Marshallow

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Henry Marshallow,Pies,3.5,4.0,2.5,4.5
Inky Rutledge,Lift,2.0,5.5,3.0,3.0
Cory Twelve,Flowers,3.5,5.0,4.0,3.0
Sixpack Santiago,Dale,2.0,5.0,4.5,3.0
Ortiz Lopez,Georgias,4.5,4.5,3.5,3.5
Howell Rocha,Dale,3.0,4.5,3.0,3.0
Liam Snail,Dale,3.0,4.5,3.5,3.0
Patchwork Southwick,Worms,3.5,4.5,2.5,3.5
Brock Forbes,Flowers,2.0,4.5,3.5,4.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Inky Rutledge,1.092614,1.515457,0.864538,0.624700,1.113789,0.950643,1.082086
Cory Twelve,1.007604,1.147990,1.086700,0.720332,0.619876,1.098161,0.291766
Sixpack Santiago,0.969910,0.765921,1.199201,1.531857,0.747720,0.826116,0.559834
Ortiz Lopez,0.932742,1.144359,0.734966,0.897736,1.103488,0.739566,1.057737
Howell Rocha,0.932527,1.014457,0.965196,0.777048,0.895984,0.570245,0.505460
Liam Snail,0.927995,1.032431,0.858471,0.775995,1.083139,1.016481,0.996009
Patchwork Southwick,0.925359,1.421321,0.564919,1.332899,0.467677,1.374752,0.877700
Brock Forbes,0.911134,0.851020,1.196748,0.890476,0.580398,0.607876,0.990014
Miguel James,0.897735,1.259409,0.825396,0.491799,0.882421,0.333087,0.715153


####Elvis Figueroa

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Elvis Figueroa,Pies,4.0,6.0,2.5,5.0
Yummy Elliott,Tacos,4.0,5.5,4.0,4.5
Rigby Friedrich,Georgias,4.5,5.0,4.5,4.5
Qais Dogwalker,Dale,3.5,5.0,5.0,4.5
Cory Twelve,Flowers,3.5,5.0,4.0,3.0
Peanutiel Duffy,Firefighters,4.5,5.0,3.0,4.0
Walton Sports,Jazz Hands,2.5,5.0,4.0,5.5
Yurts Buttercup,Georgias,4.5,4.5,3.5,4.0
Ortiz Lopez,Georgias,4.5,4.5,3.5,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Elvis Figueroa,1.173083,1.217065,1.206718,1.134739,0.851240,0.514752,0.772422
Yummy Elliott,1.112591,0.963827,1.310590,1.291143,0.788595,1.098279,1.079743
Rigby Friedrich,1.030718,1.215808,1.022046,0.550512,1.213816,0.786244,0.656845
Qais Dogwalker,1.016129,0.958834,1.026445,1.223849,1.039135,0.739043,0.469360
Cory Twelve,1.007604,1.147990,1.086700,0.720332,0.619876,1.098161,0.291766
Peanutiel Duffy,1.002021,1.232004,1.024517,0.878610,0.445467,0.625876,0.899161
Walton Sports,0.973727,1.259897,0.798052,0.686300,1.135276,0.722438,1.201971
Yurts Buttercup,0.935903,1.060457,1.153611,0.701502,0.346840,1.287475,0.634202
Ortiz Lopez,0.932742,1.144359,0.734966,0.897736,1.103488,0.739566,1.057737
Slosh Truk,0.885440,0.822684,1.116815,0.942004,0.652876,0.513286,0.661398


####Tiana Takahashi

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Tiana Takahashi,Pies,1.5,4.0,3.0,3.5
Inky Rutledge,Lift,2.0,5.5,3.0,3.0
Silvia Rugrat,Wild Wings,3.0,5.0,2.0,2.5
Bennett Bluesky,Spies,2.0,5.0,1.0,2.5
Cudi Di Batterino,Lift,2.0,5.0,1.5,3.5
Jaylen Hotdogfingers,Flowers,2.5,4.5,2.0,2.0
Howell Rocha,Dale,3.0,4.5,3.0,3.0
Wyatt Mason IV,Georgias,1.0,4.5,2.0,4.0
Erin Jesaulenko,Georgias,1.0,4.5,2.0,4.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,0.776416,0.947905,0.999000,0.997307,0.116079,0.665963,0.999000
Inky Rutledge,1.092614,1.515457,0.864538,0.624700,1.113789,0.950643,1.082086
Silvia Rugrat,1.039169,1.071044,1.229974,0.662651,0.800881,1.233076,1.361413
Bennett Bluesky,1.028889,1.259005,1.010755,0.569057,0.914312,1.108024,1.033354
Cudi Di Batterino,0.956106,0.964983,0.954493,1.308338,0.581326,1.245921,0.583496
Jaylen Hotdogfingers,0.937526,0.931926,1.068898,0.882210,0.686428,1.020742,1.009098
Howell Rocha,0.932527,1.014457,0.965196,0.777048,0.895984,0.570245,0.505460
Wyatt Mason IV,0.909013,0.859159,1.110696,0.885281,0.977619,0.194373,0.879172
Erin Jesaulenko,0.884616,1.145224,1.004160,0.330078,0.842915,0.705821,0.611776
Jacoby Podcast,0.866892,0.752357,1.071733,0.783826,1.071267,1.056401,0.413332


### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [7]:
shadows = real_pies.bullpen + real_pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

display(get_stars([worst_batter] + best_batting_shadows))
display(set_heatmap(get_batting_stlats([worst_batter] + best_batting_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Usurper Violet,3.0,3.0,1.5,4.0
Haruta Byrd,3.5,2.5,2.5,1.5
Javier Lotus,3.5,1.0,3.5,4.0
August Obrien,3.5,1.0,3.5,2.0
Sheev Shriffle,3.5,2.0,2.0,1.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958
Sheev Shriffle,0.670591,0.984880,0.610356,0.221637,0.482530,0.602199,0.774400,0.229204,0.553719


**Pitching:**

In [8]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

display(get_stars([worst_pitcher] + best_pitching_shadows))
display(set_heatmap(get_pitching_stlats([worst_pitcher] + best_pitching_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,4.0,2.5,4.5
Lucy Tokkan,2.0,3.5,1.0,3.0
Doc Anice,3.0,3.5,3.0,4.5
Twofurious Puddles,2.5,3.5,2.0,3.5
Case Lancaster,0.0,3.5,2.0,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Doc Anice,0.734279,1.076739,0.487964,0.959145,0.632001,0.765347,0.305586
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894
Case Lancaster,0.658365,0.867044,0.829197,0.190119,0.825333,0.868251,0.082117


### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving our best hitting pitcher or best pitching hitter into their preferred
position.

**Batting**

In [9]:
bhp = best_hitting_pitcher(real_pies)
bph = best_pitching_hitter(real_pies)

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "bench")))

Moving Elvis Figueroa to the Lineup

,Pies
batting_change,-1.9663%
pitching_change,-13.3573%
baserunning_change,-2.6521%
defense_change,+2.1752%


Moving Usurper Violet to the Shadows

,Pies
batting_change,+5.2382%
pitching_change,+0.0000%
baserunning_change,+5.6263%
defense_change,-0.1989%


**Pitching**

In [10]:
display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "bullpen")))

Moving Ruslan Greatness to the Rotation

,Pies
batting_change,-0.0673%
pitching_change,-2.1649%
baserunning_change,-4.1903%
defense_change,-3.5101%


Moving Henry Marshallow to the Shadows

,Pies
batting_change,+0.0000%
pitching_change,+6.8813%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Item Move
**Move an Item from your Team to a Player on your Team, repairing it fully.**

See below for a list of all non-base items available to swap. Items can be moved to any player, regardless of if they
have an item or not. Items that break or are dropped will likely still be applicable if the will vote is submitted prior
to the item going away.

In [11]:
all_items = []
for p in pies.lineup + pies.rotation + pies.bench + pies.bullpen:
    for item in p.items:
        if len(item.adjustments) > 1:
            all_items.append((p, item))

table = []
for p, i in all_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Wielder": p.name, "Max Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)


,Wielder,Max Durability,Modifications
Rubber Ring of Fourtitude,Eduardo Woodman,3,[The Fourth Strike]
Leg Shoes,Beasley Day,2,[]
Metaphorical Blood Glove,Huber Frumple,3,[]
Gravity Socks,Ruslan Greatness,2,[Gravity]
Holey Travelling Bat,Lang Richardson,3,[Traveling]
Noise-Cancelling Jersey of Charisma,Usurper Violet,3,[Soundproof]
Hearty Cool Rock Bat,Henry Marshallow,5,[]
Hard Blood Jersey,Tiana Takahashi,3,[]


### Shadow Infuse
**Boost a Player in your Team's Shadows by 15% to 30% in a Stat Category of your choice.**

See below for how this would affect various team members at the 2 extreme percentages. This will only affect one player.

**15% Change:**

In [12]:
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.15, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.15, shadows=True)), maxVal=1))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,3.192006,0.838714
August Obrien,3.408347,4.236526,0.828180
Mcfarland Vargas,1.383943,2.402244,1.018300
Twofurious Puddles,2.635533,3.508269,0.872736
Mummy Cantburn,1.390958,2.397999,1.007041
Doginic Burgertoes,2.079285,3.064023,0.984738
Nerd Pacheco,0.270478,1.099715,0.829238
Yusef Puddles,2.390547,3.272644,0.882097
Sheev Shriffle,3.352957,4.123714,0.770757
Adeline Hammer,1.095992,2.164572,1.068580


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.638401,0.452171,0.858122,1.101649,0.252259,0.125678,0.482170,0.010000,0.693764
August Obrien,0.847305,0.791217,1.077569,1.063891,0.642708,0.773854,0.706849,0.010000,0.613958
Mcfarland Vargas,0.480449,0.267084,0.926150,0.457714,0.325366,0.834279,0.567832,0.010000,0.701343
Twofurious Puddles,0.701654,0.727625,0.630623,1.007061,1.126273,0.662102,0.159182,0.010000,0.624513
Mummy Cantburn,0.479600,0.245592,0.726609,1.097265,0.675890,0.766564,0.162553,0.010000,1.077160
Doginic Burgertoes,0.612805,0.326724,1.134426,0.255629,1.084521,0.062919,0.188098,0.611237,0.454905
Nerd Pacheco,0.219943,0.191707,0.202607,0.167167,0.182564,0.794911,0.389614,0.829935,0.177951
Yusef Puddles,0.654529,1.218932,0.373433,0.852603,0.341137,0.469418,0.301715,0.010000,1.200129
Sheev Shriffle,0.824743,1.134880,0.760356,0.371637,0.632530,0.452199,0.924400,0.079204,0.703719
Adeline Hammer,0.432914,0.194688,0.556439,0.889264,1.124677,0.576302,0.533623,0.318421,0.433524


In [13]:
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.15, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.15, shadows=True)), maxVal=1))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Adrian Melon,0.939306,2.013364,1.074059
August Obrien,1.045544,1.841553,0.796009
Mcfarland Vargas,1.501794,2.384134,0.882341
Twofurious Puddles,3.474609,4.330050,0.855440
Mummy Cantburn,2.326166,3.166361,0.840195
Doginic Burgertoes,1.808231,2.676227,0.867997
Nerd Pacheco,0.334608,0.999587,0.664979
Yusef Puddles,2.508852,3.348354,0.839502
Sheev Shriffle,1.942316,2.803450,0.861135
Adeline Hammer,0.915392,2.064447,1.149055


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Adrian Melon,0.402673,0.342291,1.106712,0.171132,0.242483,0.736090,0.879182
August Obrien,0.368311,0.386723,0.411477,1.037830,0.247743,0.250188,0.472708
Mcfarland Vargas,0.476827,0.337950,0.992297,0.436164,0.498503,0.960603,0.499185
Twofurious Puddles,0.866010,0.961110,0.800034,0.957712,0.868615,0.566240,0.875894
Mummy Cantburn,0.633272,0.773505,0.742185,0.389305,0.587026,0.562072,1.122225
Doginic Burgertoes,0.535245,0.739539,0.364470,1.063112,0.461613,0.911430,0.170448
Nerd Pacheco,0.199917,0.269127,0.275662,0.206284,0.206083,0.176155,0.191882
Yusef Puddles,0.669671,0.719080,0.627310,1.183877,0.475880,0.975805,0.767095
Sheev Shriffle,0.560690,0.413763,0.959104,0.629938,0.594447,1.027531,0.791488
Adeline Hammer,0.412889,0.921047,0.191792,0.687204,0.288224,0.914063,0.959582


**30% Change:**

In [14]:
infuse_lineup_table, _, _ = improve_team_batting_table(real_pies, 0.30, shadows=True)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.30, shadows=True)), maxVal=1))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,3.949166,1.595874
August Obrien,3.408347,4.978021,1.569675
Mcfarland Vargas,1.383943,3.227008,1.843065
Twofurious Puddles,2.635533,4.249688,1.614155
Mummy Cantburn,1.390958,3.221026,1.830069
Doginic Burgertoes,2.079285,3.886515,1.807229
Nerd Pacheco,0.270478,1.867448,1.596971
Yusef Puddles,2.390547,4.073123,1.682575
Sheev Shriffle,3.352957,4.857104,1.504147
Adeline Hammer,1.095992,3.000781,1.904789


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.789833,0.602171,1.008122,1.251649,0.402259,0.010000,0.632170,0.010000,0.843764
August Obrien,0.995604,0.941217,1.227569,1.213891,0.792708,0.623854,0.856849,0.010000,0.763958
Mcfarland Vargas,0.645402,0.417084,1.076150,0.607714,0.475366,0.684279,0.717832,0.010000,0.851343
Twofurious Puddles,0.849938,0.877625,0.780623,1.157061,1.276273,0.512102,0.309182,0.010000,0.774513
Mummy Cantburn,0.644205,0.395592,0.876609,1.247265,0.825890,0.616564,0.312553,0.010000,1.227160
Doginic Burgertoes,0.777303,0.476724,1.284426,0.405629,1.234521,0.010000,0.338098,0.461237,0.604905
Nerd Pacheco,0.373490,0.341707,0.352607,0.317167,0.332564,0.644911,0.539614,0.679935,0.327951
Yusef Puddles,0.814625,1.368932,0.523433,1.002603,0.491137,0.319418,0.451715,0.010000,1.350129
Sheev Shriffle,0.971421,1.284880,0.910356,0.521637,0.782530,0.302199,1.074400,0.010000,0.853719
Adeline Hammer,0.600156,0.344688,0.706439,1.039264,1.274677,0.426302,0.683623,0.168421,0.583524


In [15]:
infuse_rotation_table, _, _ = improve_team_pitching_table(real_pies, 0.30, shadows=True)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.30, shadows=True)), maxVal=1))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Adrian Melon,0.939306,2.943382,2.004077
August Obrien,1.045544,2.659549,1.614005
Mcfarland Vargas,1.501794,3.264392,1.762598
Twofurious Puddles,3.474609,5.209376,1.734766
Mummy Cantburn,2.326166,4.024465,1.698299
Doginic Burgertoes,1.808231,3.547943,1.739712
Nerd Pacheco,0.334608,1.727941,1.393333
Yusef Puddles,2.508852,4.211538,1.702686
Sheev Shriffle,1.942316,3.675139,1.732824
Adeline Hammer,0.915392,3.024460,2.109067


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Adrian Melon,0.588676,0.492291,1.256712,0.321132,0.392483,0.886090,1.029182
August Obrien,0.531910,0.536723,0.561477,1.187830,0.397743,0.400188,0.622708
Mcfarland Vargas,0.652878,0.487950,1.142297,0.586164,0.648503,1.110603,0.649185
Twofurious Puddles,1.041875,1.111110,0.950034,1.107712,1.018615,0.716240,1.025894
Mummy Cantburn,0.804893,0.923505,0.892185,0.539305,0.737026,0.712072,1.272225
Doginic Burgertoes,0.709589,0.889539,0.514470,1.213112,0.611613,1.061430,0.320448
Nerd Pacheco,0.345588,0.419127,0.425662,0.356284,0.356083,0.326155,0.341882
Yusef Puddles,0.842308,0.869080,0.777310,1.333877,0.625880,1.125805,0.917095
Sheev Shriffle,0.735028,0.563763,1.109104,0.779938,0.744447,1.177531,0.941488
Adeline Hammer,0.604892,1.071047,0.341792,0.837204,0.438224,1.064063,1.109582


### Shadow Revoke
**Revoke a Player from your Team's Shadows.**

This would remove a player from our shadows by moving them to another team, without providing a replacement.

### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [16]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

,mods
Beasley Day,[Minimalist]
Huber Frumple,[Seeker]
Ruslan Greatness,[Friend of Crows]
Jaxon Buckley,[Fire Eater]
Peanut Holloway,"[Honey Roasted, Attractor]"
Henry Marshallow,[Maximalist]
Elvis Figueroa,[Triple Threat]


### Alternate Trust
**Call in an Alternate for a Player on your Team, with a mystery Benefit.**

An Alternate is a player reroll and the Benefit is an unknown modification. Alternates were changed for S18 and onward
to always result in a player with the same combined stars. This would be useful for rerolling a player that has a
less-optimal player attribute distribution.

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### Now Hiring
**Recruit a Random Player with the Roamin' Modification to your Team.**

See below for list of all active players with Roamin'. This will not remove an existing player from our team.

In [17]:
league = League.load()
divisions = league.subleagues
all_teams = []
for division in divisions.values():
    all_teams.extend(list(division.teams.values()))

In [18]:
siphons = []
for team in all_teams:
    siphons.extend([x for x in team.lineup + team.rotation if "WANDERER" in x._perm_attr_ids])
get_stars(siphons, include_team=True)

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Mags Banananana,Lift,1.5,3.5,3.0,2.5
NaN,Flowers,2.0,1.0,1.5,1.0
Jasper Blather,Spies,2.5,3.0,2.5,3.5
S--w Bri--s,Worms,1.5,0.0,2.0,1.0
Alejandro Leaf,Shoe Thieves,2.5,3.0,4.5,3.5
Evelton McBlase II,Magic,2.0,3.0,3.0,4.5


### Super Roamin' Fifth Base
**Give the best Baserunner in your Team's Lineup the Legendary Super Roamin' Fifth Base.**

Unknown, likely an Item with both Super Roamin' and Fifth Base. A Super Roamin' player will likely immediately leave
our team.

In [19]:
p = pies.lineup.copy()
p.sort(key=lambda x: x.baserunning_rating, reverse=True)
display(Markdown(f"Will be given to {p[0].name}."))

Will be given to Ruslan Greatness.

### Fax Preparation
**Move the Best Pitcher in your Team's Lineup to your Shadows.**

See below for analysis.

In [20]:
bph = best_pitching_hitter(pies)
display(Markdown(f"**Moving {bph.name} to the Shadows**"))

pandas.DataFrame(remove_player(pies, bph))

**Moving Ruslan Greatness to the Shadows**

,Pies
batting_change,-0.0673%
pitching_change,+0.0000%
baserunning_change,-4.1903%
defense_change,-3.5101%


### Hits Keep Coming
**Sort your Lineup in order of Tragicness.**

See below for new lineup order.

In [21]:
new_lineup = pies.lineup.copy()
new_lineup.sort(key=lambda x: x.tragicness, reverse=True)
get_stars(new_lineup)

,Batting,Pitching,Baserunning,Defense
Peanut Holloway,6.0,4.0,4.0,5.5
Usurper Violet,3.0,3.0,1.5,4.0
Huber Frumple,4.5,2.0,4.0,3.0
Eduardo Woodman,5.5,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,3.5,3.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5


### Outdoorsy
**A random Player on your Team will play better in Grandiose Ballparks and worse in Ungrandiose Ballparks.**

This will likely cause a player to perform better in Stadiums with Grandiosity >0.5 and worst in Stadiums with
Grandiosity <0.5.

In [22]:
stadiums = Stadium.load_all()
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Grandiosity": s.grandiosity}, name=s.team_id.nickname))
table = table.sort_values(by="Grandiosity")
set_heatmap(table, maxVal=1)

,Grandiosity
Steaks,0.246684
Pies,0.351436
Flowers,0.358945
Fridays,0.410747
Mechanics,0.444729
Breath Mints,0.451429
Lovers,0.469349
Wild Wings,0.504088
Tacos,0.510360
Firefighters,0.511205


### Gaudy
**A random Player on your Team will play better in Ballparks with more Modifications.**

See below for list of Stadiums and their number of modifications.

In [23]:
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Number of Mods": len(s.mods)}, name=s.team_id.nickname))
table.sort_values(by="Number of Mods")

,Number of Mods
Steaks,3.0
Lovers,5.0
Pies,5.0
Wild Wings,6.0
Worms,6.0
Fridays,6.0
Millennials,7.0
Dale,7.0
Sunbeams,7.0
Jazz Hands,7.0


### Cluttered
**A random Player on your Team will play better in Filthier Ballparks.**

See below for list of Filthiest Stadiums.

In [24]:
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Filthiness": s.filthiness}, name=s.team_id.nickname))
table = table.sort_values(by="Filthiness")
set_heatmap(table, maxVal=1)

,Filthiness
Moist Talkers,0.006641
Sunbeams,0.009690
Garages,0.011321
Georgias,0.067136
Crabs,0.074568
Lovers,0.079215
Breath Mints,0.090463
Magic,0.094152
Millennials,0.100622
Mechanics,0.107913


### Guarded
**A random Player on your Team will play better in Fortified Ballparks and worse in Unfortified Ballparks.**

This will likely cause a player to perform better in Stadiums with Fortification >0.5 and worst in Stadiums with
Fortification <0.5.

In [25]:
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Fortification": s.fortification}, name=s.team_id.nickname))
table = table.sort_values(by="Fortification")
set_heatmap(table, maxVal=1)

,Fortification
Tigers,0.100083
Jazz Hands,0.268921
Dale,0.271895
Breath Mints,0.300756
Tacos,0.327867
Fridays,0.375367
Pies,0.377286
Sunbeams,0.405864
Moist Talkers,0.424108
Spies,0.492716


### Practice
**Your Team's Lineup and Rotation will have their Defense Boosted from 1% to 4%.**

**Your Team's Lineup will have their Baserunning Boosted from 1% to 4%.**

**Your Team's Rotation will have their Pitching Boosted from 1% to 4%.**

**Your Team's Lineup will have their Hitting Boosted from 1% to 4%.**

These blessings increase the team-wide stats. See below for worst and best case rolls.

#### Batting Practice

**Worst Case**

In [26]:
table, total, avg = improve_team_batting_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.01)), maxVal=1))

Change in Total stars: 0.38070165876132833

Change in Average stars: 0.04758770734516604

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,5.632212,5.678029,0.045817
Beasley Day,4.522972,4.569600,0.046628
Huber Frumple,4.435537,4.485968,0.050432
Ruslan Greatness,4.755482,4.799493,0.044011
Jaxon Buckley,6.592812,6.632101,0.039289
Lang Richardson,3.162214,3.212664,0.050450
Peanut Holloway,3.570437,3.618452,0.048015
Usurper Violet,2.898577,2.954638,0.056061


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.135606,1.355315,1.162985,0.980888,0.886160,0.273687,0.759895,0.010000,1.035183
Beasley Day,0.913920,1.055160,0.914890,0.622953,0.717377,0.201371,0.743784,0.010000,1.019991
Huber Frumple,0.897194,0.544025,1.355741,0.969816,1.252462,0.178405,0.718998,0.010000,1.148288
Ruslan Greatness,0.959899,0.632589,1.155689,1.579998,1.480457,0.010000,1.321486,0.010000,0.693552
Jaxon Buckley,1.326420,1.577382,1.052101,1.756709,1.940045,0.010000,1.996793,0.010000,1.403504
Lang Richardson,0.642533,0.822722,0.410621,0.632843,0.951821,0.195246,0.508400,0.010000,0.561015
Peanut Holloway,0.723690,0.666788,0.759014,0.660239,0.765549,0.224531,0.387807,0.197685,0.321857
Usurper Violet,0.590928,0.904648,0.303503,1.032726,0.879795,0.559699,0.293561,0.090000,0.497176


**Best Case**

In [27]:
table, total, avg = improve_team_batting_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.04)), maxVal=1))


Change in Total stars: 1.5392904026879353

Change in Average stars: 0.1924113003359919

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,5.632212,5.816714,0.184502
Beasley Day,4.522972,4.710365,0.187393
Huber Frumple,4.435537,4.637024,0.201488
Ruslan Greatness,4.755482,4.938496,0.183014
Jaxon Buckley,6.592812,6.760354,0.167542
Lang Richardson,3.162214,3.363875,0.201662
Peanut Holloway,3.570437,3.762139,0.191702
Usurper Violet,2.898577,3.120565,0.221988


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.163343,1.385315,1.192985,1.010888,0.916160,0.243687,0.789895,0.010000,1.065183
Beasley Day,0.942073,1.085160,0.944890,0.652953,0.747377,0.171371,0.773784,0.010000,1.049991
Huber Frumple,0.927405,0.574025,1.385741,0.999816,1.282462,0.148405,0.748998,0.010000,1.178288
Ruslan Greatness,0.987699,0.662589,1.185689,1.609998,1.510457,0.010000,1.351486,0.010000,0.723552
Jaxon Buckley,1.352071,1.607382,1.082101,1.786709,1.970045,0.010000,2.026793,0.010000,1.433504
Lang Richardson,0.672775,0.852722,0.440621,0.662843,0.981821,0.165246,0.538400,0.010000,0.591015
Peanut Holloway,0.752428,0.696788,0.789014,0.690239,0.795549,0.194531,0.417807,0.167685,0.351857
Usurper Violet,0.624113,0.934648,0.333503,1.062726,0.909795,0.529699,0.323561,0.060000,0.527176


#### Pitching Practice

**Worst Case**

In [28]:
table, total, avg = improve_team_pitching_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.01)), maxVal=1))

Change in Total stars: 0.19908361891920778

Change in Average stars: 0.06636120630640259

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,3.903660,0.062110
Elvis Figueroa,5.865415,5.926482,0.061067
Tiana Takahashi,3.882081,3.957987,0.075906


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.780732,0.501270,1.181077,1.438524,0.832605,0.818351,0.886106
Elvis Figueroa,1.185296,1.227065,1.216718,1.144739,0.861240,0.524752,0.782422
Tiana Takahashi,0.791597,0.957905,1.009000,1.007307,0.126079,0.675963,1.009000


**Best Case**

In [29]:
table, total, avg = improve_team_pitching_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.04)), maxVal=1))

Change in Total stars: 0.7885503890993713

Change in Average stars: 0.26285012969979044

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,4.090007,0.248457
Elvis Figueroa,5.865415,6.110114,0.244699
Tiana Takahashi,3.882081,4.177476,0.295395


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.818001,0.531270,1.211077,1.468524,0.862605,0.848351,0.916106
Elvis Figueroa,1.222023,1.257065,1.246718,1.174739,0.891240,0.554752,0.812422
Tiana Takahashi,0.835495,0.987905,1.039000,1.037307,0.156079,0.705963,1.039000


#### Running Practice

**Worst Case**

In [30]:
table, total, avg = improve_team_baserunning_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.01)), maxVal=1))

Change in Total stars: 0.5571967987386641

Change in Average stars: 0.06964959984233302

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Eduardo Woodman,3.665244,3.712517,0.047273
Beasley Day,3.626384,3.674067,0.047683
Huber Frumple,4.153972,4.200001,0.046029
Ruslan Greatness,5.110398,5.158317,0.047918
Jaxon Buckley,3.487603,3.535628,0.048025
Lang Richardson,3.581538,3.629955,0.048416
Peanut Holloway,1.789429,1.851987,0.062557
Usurper Violet,1.674574,1.883868,0.209294


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Eduardo Woodman,0.742503,0.664141,1.087257,0.562728,0.753821,0.854641
Beasley Day,0.734813,0.629583,0.679022,1.199892,0.602646,0.944967
Huber Frumple,0.840000,0.798282,0.747669,0.898626,0.968223,0.829371
Ruslan Greatness,1.031663,1.385092,0.612818,0.703571,1.009000,0.615824
Jaxon Buckley,0.707126,0.699885,1.202203,0.516722,0.523522,0.572351
Lang Richardson,0.725991,0.752498,0.431602,0.855647,0.456945,0.998950
Peanut Holloway,0.370397,0.338968,0.082877,0.289784,0.594838,0.760281
Usurper Violet,0.376774,0.839727,1.006871,0.232526,0.023191,0.025431


**Best Case**

In [31]:
table, total, avg = improve_team_baserunning_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.04)), maxVal=1))

Change in Total stars: 1.9903956831794272

Change in Average stars: 0.2487994603974284

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Eduardo Woodman,3.665244,3.853835,0.188591
Beasley Day,3.626384,3.816564,0.190180
Huber Frumple,4.153972,4.337740,0.183768
Ruslan Greatness,5.110398,5.301372,0.190974
Jaxon Buckley,3.487603,3.679092,0.191489
Lang Richardson,3.581538,3.774434,0.192895
Peanut Holloway,1.789429,2.030326,0.240897
Usurper Violet,1.674574,2.286176,0.611602


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Eduardo Woodman,0.770767,0.694141,1.117257,0.592728,0.783821,0.884641
Beasley Day,0.763313,0.659583,0.709022,1.229892,0.632646,0.974967
Huber Frumple,0.867548,0.828282,0.777669,0.928626,0.998223,0.859371
Ruslan Greatness,1.060274,1.415092,0.642818,0.733571,1.039000,0.645824
Jaxon Buckley,0.735818,0.729885,1.232203,0.546722,0.553522,0.602351
Lang Richardson,0.754887,0.782498,0.461602,0.885647,0.486945,1.028950
Peanut Holloway,0.406065,0.368968,0.112877,0.319784,0.624838,0.790281
Usurper Violet,0.457235,0.869727,1.036871,0.262526,0.053191,0.055431


#### Defense Practice

**Worst Case**

In [32]:
table, total, avg = improve_team_defense_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.01)), maxVal=1))

Change in Total stars: 0.469196171998425

Change in Average stars: 0.04265419745440227

,old_defense_stars,new_defense_stars,change_in_defense_stars
Eduardo Woodman,3.251824,3.297948,0.046124
Beasley Day,3.048787,3.095186,0.046398
Huber Frumple,3.055612,3.101891,0.046279
Ruslan Greatness,5.060321,5.095564,0.035244
Jaxon Buckley,3.288755,3.331888,0.043132
Lang Richardson,3.709204,3.753803,0.044599
Peanut Holloway,2.965752,3.013533,0.047781
Usurper Violet,3.901407,3.942486,0.041078
Henry Marshallow,4.667232,4.705624,0.038392
Elvis Figueroa,4.810661,4.847193,0.036532


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Eduardo Woodman,0.659590,0.387478,0.363459,0.844861,0.962460,0.966417
Beasley Day,0.619037,0.365386,0.408364,1.314772,0.467722,0.603574
Huber Frumple,0.620378,0.514837,0.312478,0.500571,1.057842,0.616171
Ruslan Greatness,1.019113,1.180016,0.904059,0.913316,0.858210,1.354688
Jaxon Buckley,0.666378,0.646443,0.480729,0.448125,0.937871,0.425402
Lang Richardson,0.750761,1.053616,0.950524,0.350104,0.400675,0.404329
Peanut Holloway,0.602707,0.358274,0.315188,0.981296,0.787629,0.641750
Usurper Violet,0.788497,1.074183,0.750791,0.543876,0.400291,0.656039
Henry Marshallow,0.941125,0.876120,1.315255,0.771818,0.460244,1.155636
Elvis Figueroa,0.969439,1.123911,1.148358,0.654382,0.693850,0.969366


**Best Case**

In [33]:
table, total, avg = improve_team_defense_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.04)), maxVal=1))

Change in Total stars: 1.8566657848262063

Change in Average stars: 0.1687877986205642

,old_defense_stars,new_defense_stars,change_in_defense_stars
Eduardo Woodman,3.251824,3.433887,0.182063
Beasley Day,3.048787,3.232098,0.183310
Huber Frumple,3.055612,3.238288,0.182676
Ruslan Greatness,5.060321,5.200625,0.140304
Jaxon Buckley,3.288755,3.459631,0.170876
Lang Richardson,3.709204,3.885293,0.176089
Peanut Holloway,2.965752,3.154084,0.188332
Usurper Violet,3.901407,4.064229,0.162822
Henry Marshallow,4.667232,4.819612,0.152381
Elvis Figueroa,4.810661,4.955952,0.145291


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Eduardo Woodman,0.686777,0.417478,0.393459,0.874861,0.992460,0.996417
Beasley Day,0.646420,0.395386,0.438364,1.344772,0.497722,0.633574
Huber Frumple,0.647658,0.544837,0.342478,0.530571,1.087842,0.646171
Ruslan Greatness,1.040125,1.210016,0.934059,0.943316,0.888210,1.384688
Jaxon Buckley,0.691926,0.676443,0.510729,0.478125,0.967871,0.455402
Lang Richardson,0.777059,1.083616,0.980524,0.380104,0.430675,0.434329
Peanut Holloway,0.630817,0.388274,0.345188,1.011296,0.817629,0.671750
Usurper Violet,0.812846,1.104183,0.780791,0.573876,0.430291,0.686039
Henry Marshallow,0.963922,0.906120,1.345255,0.801818,0.490244,1.185636
Elvis Figueroa,0.991190,1.153911,1.178358,0.684382,0.723850,0.999366


### Shadow Boosts
**Your Team's Shadows will have their Hitting Boosted from -5% to 20%.**

**Your Team's Shadows will have their Pitching Boosted from -5% to 20%.**

**Your Team's Shadows will have their Baserunning Boosted from -5% to 20%.**

**Your Team's Shadows will have their Defense Boosted from -5% to 20%.**

These blessings increase the team-wide stats or our shadow players. See below for worst and best case rolls. This
assumes that it will only boost the respective shadow positions, so batting will only affect the bench, etc.

#### Shadow of the Bats

**Worst Case**

In [34]:
table, total, avg = improve_team_batting_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, -0.05, shadows=True)), maxVal=1))

Change in Total stars: -5.194715726701101

Change in Average stars: -0.2734060908790053

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,2.023988,-0.329303
August Obrien,3.408347,3.035183,-0.373163
Mcfarland Vargas,1.383943,1.043400,-0.340543
Twofurious Puddles,2.635533,2.401668,-0.233864
Mummy Cantburn,1.390958,0.976989,-0.413969
Doginic Burgertoes,2.079285,1.669925,-0.409360
Nerd Pacheco,0.270478,0.073203,-0.197275
Yusef Puddles,2.390547,2.059069,-0.331479
Sheev Shriffle,3.352957,3.082041,-0.270916
Adeline Hammer,1.095992,0.605714,-0.490278


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.404798,0.252171,0.658122,0.901649,0.052259,0.325678,0.282170,0.150000,0.493764
August Obrien,0.607037,0.591217,0.877569,0.863891,0.442708,0.973854,0.506849,0.150000,0.413958
Mcfarland Vargas,0.208680,0.067084,0.726150,0.257714,0.125366,0.990000,0.367832,0.150000,0.501343
Twofurious Puddles,0.480334,0.527625,0.430623,0.807061,0.926273,0.862102,0.010000,0.150000,0.424513
Mummy Cantburn,0.195398,0.045592,0.526609,0.897265,0.475890,0.966564,0.010000,0.150000,0.877160
Doginic Burgertoes,0.333985,0.126724,0.934426,0.055629,0.884521,0.262919,0.010000,0.811237,0.254905
Nerd Pacheco,0.014641,0.010000,0.010000,0.010000,0.010000,0.990000,0.189614,0.990000,0.010000
Yusef Puddles,0.411814,1.018932,0.173433,0.652603,0.141137,0.669418,0.101715,0.062283,1.000129
Sheev Shriffle,0.616408,0.934880,0.560356,0.171637,0.432530,0.652199,0.724400,0.279204,0.503719
Adeline Hammer,0.121143,0.010000,0.356439,0.689264,0.924677,0.776302,0.333623,0.518421,0.233524


**Best Case**

In [35]:
table, total, avg = improve_team_batting_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.20, shadows=True)), maxVal=1))


Change in Total stars: 22.706511706454677

Change in Average stars: 1.1950795634976146

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,3.451552,1.098261
August Obrien,3.408347,4.488341,1.079995
Mcfarland Vargas,1.383943,2.686492,1.302549
Twofurious Puddles,2.635533,3.759716,1.124183
Mummy Cantburn,1.390958,2.681958,1.291000
Doginic Burgertoes,2.079285,3.353370,1.274085
Nerd Pacheco,0.270478,1.359404,1.088926
Yusef Puddles,2.390547,3.545751,1.155204
Sheev Shriffle,3.352957,4.372160,1.019204
Adeline Hammer,1.095992,2.454162,1.358170


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.690310,0.502171,0.908122,1.151649,0.302259,0.075678,0.532170,0.010000,0.743764
August Obrien,0.897668,0.841217,1.127569,1.113891,0.692708,0.723854,0.756849,0.010000,0.663958
Mcfarland Vargas,0.537298,0.317084,0.976150,0.507714,0.375366,0.784279,0.617832,0.010000,0.751343
Twofurious Puddles,0.751943,0.777625,0.680623,1.057061,1.176273,0.612102,0.209182,0.010000,0.674513
Mummy Cantburn,0.536392,0.295592,0.776609,1.147265,0.725890,0.716564,0.212553,0.010000,1.127160
Doginic Burgertoes,0.670674,0.376724,1.184426,0.305629,1.134521,0.012919,0.238098,0.561237,0.504905
Nerd Pacheco,0.271881,0.241707,0.252607,0.217167,0.232564,0.744911,0.439614,0.779935,0.227951
Yusef Puddles,0.709150,1.268932,0.423433,0.902603,0.391137,0.419418,0.351715,0.010000,1.250129
Sheev Shriffle,0.874432,1.184880,0.810356,0.421637,0.682530,0.402199,0.974400,0.029204,0.753719
Adeline Hammer,0.490832,0.244688,0.606439,0.939264,1.174677,0.526302,0.583623,0.268421,0.483524


#### Fax Numbers

**Worst Case**

In [36]:
table, total, avg = improve_team_pitching_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, -0.05, shadows=True)), maxVal=1))

Change in Total stars: -5.370727258022761

Change in Average stars: -0.2826698556854085

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Adrian Melon,0.939306,0.652527,-0.286778
August Obrien,1.045544,0.773667,-0.271877
Mcfarland Vargas,1.501794,1.199200,-0.302593
Twofurious Puddles,3.474609,3.195225,-0.279385
Mummy Cantburn,2.326166,2.047462,-0.278705
Doginic Burgertoes,1.808231,1.513197,-0.295034
Nerd Pacheco,0.334608,0.131930,-0.202678
Yusef Puddles,2.508852,2.234300,-0.274552
Sheev Shriffle,1.942316,1.654287,-0.288029
Adeline Hammer,0.915392,0.469906,-0.445487


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Adrian Melon,0.130505,0.142291,0.906712,0.010000,0.042483,0.536090,0.679182
August Obrien,0.154733,0.186723,0.211477,0.837830,0.047743,0.050188,0.272708
Mcfarland Vargas,0.239840,0.137950,0.792297,0.236164,0.298503,0.760603,0.299185
Twofurious Puddles,0.639045,0.761110,0.600034,0.757712,0.668615,0.366240,0.675894
Mummy Cantburn,0.409492,0.573505,0.542185,0.189305,0.387026,0.362072,0.922225
Doginic Burgertoes,0.302639,0.539539,0.164470,0.863112,0.261613,0.711430,0.010000
Nerd Pacheco,0.026386,0.069127,0.075662,0.010000,0.010000,0.010000,0.010000
Yusef Puddles,0.446860,0.519080,0.427310,0.983877,0.275880,0.775805,0.567095
Sheev Shriffle,0.330857,0.213763,0.759104,0.429938,0.394447,0.827531,0.591488
Adeline Hammer,0.093981,0.721047,0.010000,0.487204,0.088224,0.714063,0.759582


**Best Case**

In [37]:
table, total, avg = improve_team_pitching_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.20, shadows=True)), maxVal=1))

Change in Total stars: 22.520029933005187

Change in Average stars: 1.1852647333160624

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Adrian Melon,0.939306,2.326798,1.387492
August Obrien,1.045544,2.111234,1.065690
Mcfarland Vargas,1.501794,2.676728,1.174935
Twofurious Puddles,3.474609,4.620631,1.146021
Mummy Cantburn,2.326166,3.450122,1.123956
Doginic Burgertoes,1.808231,2.965528,1.157297
Nerd Pacheco,0.334608,1.236132,0.901524
Yusef Puddles,2.508852,3.633514,1.124663
Sheev Shriffle,1.942316,3.092375,1.150060
Adeline Hammer,0.915392,2.391269,1.475877


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Adrian Melon,0.465360,0.392291,1.156712,0.221132,0.292483,0.786090,0.929182
August Obrien,0.422247,0.436723,0.461477,1.087830,0.297743,0.300188,0.522708
Mcfarland Vargas,0.535346,0.387950,1.042297,0.486164,0.548503,1.010603,0.549185
Twofurious Puddles,0.924126,1.011110,0.850034,1.007712,0.918615,0.616240,0.925894
Mummy Cantburn,0.690024,0.823505,0.792185,0.439305,0.637026,0.612072,1.172225
Doginic Burgertoes,0.593106,0.789539,0.414470,1.113112,0.511613,0.961430,0.220448
Nerd Pacheco,0.247226,0.319127,0.325662,0.256284,0.256083,0.226155,0.241882
Yusef Puddles,0.726703,0.769080,0.677310,1.233877,0.525880,1.025805,0.817095
Sheev Shriffle,0.618475,0.463763,1.009104,0.679938,0.644447,1.077531,0.841488
Adeline Hammer,0.478254,0.971047,0.241792,0.737204,0.338224,0.964063,1.009582


#### Shadowrun

**Worst Case**

In [38]:
table, total, avg = improve_team_baserunning_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, -0.05, shadows=True)), maxVal=1))

Change in Total stars: -5.752897135317134

Change in Average stars: -0.3027840597535334

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,2.830166,-0.286531
August Obrien,3.318856,2.997685,-0.321171
Mcfarland Vargas,3.783245,3.537400,-0.245845
Twofurious Puddles,1.817993,1.509838,-0.308155
Mummy Cantburn,2.021513,1.681256,-0.340257
Doginic Burgertoes,1.779564,1.447579,-0.331984
Nerd Pacheco,0.553414,0.277910,-0.275504
Yusef Puddles,2.938345,2.676871,-0.261474
Sheev Shriffle,2.215166,1.935936,-0.279230
Adeline Hammer,2.451014,2.195175,-0.255839


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,0.566033,0.790831,0.534640,0.435051,0.126399,0.371241
August Obrien,0.599537,0.916470,0.090389,0.651812,0.431149,0.365344
Mcfarland Vargas,0.707480,0.872765,0.507085,0.762168,0.432377,0.371246
Twofurious Puddles,0.301968,0.143390,0.328033,0.464860,0.838752,0.813079
Mummy Cantburn,0.336251,0.238250,0.782320,0.889393,0.048762,0.709435
Doginic Burgertoes,0.289516,0.272288,0.010000,0.567854,0.577448,0.843013
Nerd Pacheco,0.055582,0.072806,0.010000,0.010000,0.137564,0.010000
Yusef Puddles,0.535374,0.581219,0.199357,0.983696,0.446257,0.333273
Sheev Shriffle,0.387187,0.223349,0.568521,0.761066,0.921078,0.341838
Adeline Hammer,0.439035,0.312092,0.468388,0.618718,0.774803,0.400209


**Best Case**

In [39]:
table, total, avg = improve_team_baserunning_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.20, shadows=True)), maxVal=1))

Change in Total stars: 20.12900006599506

Change in Average stars: 1.0594210561050033

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,4.145628,1.028930
August Obrien,3.318856,4.398980,1.080124
Mcfarland Vargas,3.783245,4.734596,0.951351
Twofurious Puddles,1.817993,2.913054,1.095061
Mummy Cantburn,2.021513,3.133820,1.112307
Doginic Burgertoes,1.779564,3.023082,1.243518
Nerd Pacheco,0.553414,1.681272,1.127858
Yusef Puddles,2.938345,3.927098,0.988753
Sheev Shriffle,2.215166,3.254024,1.038858
Adeline Hammer,2.451014,3.433736,0.982722


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,0.829126,1.040831,0.784640,0.685051,0.376399,0.621241
August Obrien,0.879796,1.166470,0.340389,0.901812,0.681149,0.615344
Mcfarland Vargas,0.946919,1.122765,0.757085,1.012168,0.682377,0.621246
Twofurious Puddles,0.582611,0.393390,0.578033,0.714860,1.088752,1.063079
Mummy Cantburn,0.626764,0.488250,1.032320,1.139393,0.298762,0.959435
Doginic Burgertoes,0.604616,0.522288,0.227095,0.817854,0.827448,1.093013
Nerd Pacheco,0.336254,0.322806,0.225072,0.244925,0.387564,0.246761
Yusef Puddles,0.785420,0.831219,0.449357,1.233696,0.696257,0.583273
Sheev Shriffle,0.650805,0.473349,0.818521,1.011066,1.171078,0.591838
Adeline Hammer,0.686747,0.562092,0.718388,0.868718,1.024803,0.650209


#### Cover Letters

**Worst Case**

In [40]:
table, total, avg = improve_team_defense_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, -0.05, shadows=True)), maxVal=1))

Change in Total stars: -5.6450592197676706

Change in Average stars: -0.2971083799877721

,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,2.376623,-0.287789
August Obrien,2.035102,1.375043,-0.660059
Mcfarland Vargas,2.071032,1.401463,-0.669570
Twofurious Puddles,3.610790,3.377416,-0.233374
Mummy Cantburn,3.289650,3.069799,-0.219850
Doginic Burgertoes,1.538638,0.963733,-0.574906
Nerd Pacheco,0.500829,0.199054,-0.301775
Yusef Puddles,3.712285,3.491070,-0.221215
Sheev Shriffle,0.873827,0.946148,0.072321
Adeline Hammer,3.312777,3.070582,-0.242195


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.475325,0.138840,0.293662,0.855908,0.466713,0.886550
August Obrien,0.275009,0.010000,0.433368,0.787103,0.277928,0.602261
Mcfarland Vargas,0.280293,0.010000,0.306182,0.540229,0.691915,0.854134
Twofurious Puddles,0.675483,0.797265,0.859861,0.210410,0.605028,0.330553
Mummy Cantburn,0.613960,0.541982,0.346224,0.466128,0.575545,0.805624
Doginic Burgertoes,0.192747,0.010000,0.814391,0.365999,0.068550,0.042532
Nerd Pacheco,0.039811,0.010000,0.010000,0.010000,0.010000,0.010000
Yusef Puddles,0.698214,0.466601,0.952048,0.265498,0.591809,0.888046
Sheev Shriffle,0.189230,0.088111,0.010000,0.208516,0.413167,0.880159
Adeline Hammer,0.614116,0.441199,0.240750,0.947193,0.944874,0.755600


**Best Case**

In [41]:
table, total, avg = improve_team_defense_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.20, shadows=True)), maxVal=1))

Change in Total stars: 20.27980852851376

Change in Average stars: 1.0673583436059875

,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,3.626229,0.961817
August Obrien,2.035102,3.265688,1.230586
Mcfarland Vargas,2.071032,3.315858,1.244825
Twofurious Puddles,3.610790,4.445614,0.834824
Mummy Cantburn,3.289650,4.105055,0.815405
Doginic Burgertoes,1.538638,2.876826,1.338187
Nerd Pacheco,0.500829,1.835336,1.334507
Yusef Puddles,3.712285,4.522409,0.810124
Sheev Shriffle,0.873827,2.631009,1.757183
Adeline Hammer,3.312777,4.176898,0.864121


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.725246,0.388840,0.543662,1.105908,0.716713,1.136550
August Obrien,0.653138,0.254615,0.683368,1.037103,0.527928,0.852261
Mcfarland Vargas,0.663172,0.254401,0.556182,0.790229,0.941915,1.104134
Twofurious Puddles,0.889123,1.047265,1.109861,0.460410,0.855028,0.580553
Mummy Cantburn,0.821011,0.791982,0.596224,0.716128,0.825545,1.055624
Doginic Burgertoes,0.575365,0.247259,1.064391,0.615999,0.318550,0.292532
Nerd Pacheco,0.367067,0.230500,0.252892,0.251010,0.233923,0.222577
Yusef Puddles,0.904482,0.716601,1.202048,0.515498,0.841809,1.138046
Sheev Shriffle,0.526202,0.338111,0.203537,0.458516,0.663167,1.130159
Adeline Hammer,0.835380,0.691199,0.490750,1.197193,1.194874,1.005600


### Junk Baller
**A Random Player in your Team's Rotation will have their Unthwackability Boosted from -20% to +60%.**

Unthwackability is the player attribute most associated with increasing ground and flyouts and reducing overall hits.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [42]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"unthwackability": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,3.0,2.5,4.5
Elvis Figueroa,4.0,5.5,2.5,5.0
Tiana Takahashi,1.5,3.5,3.0,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.591594,0.291270,1.171077,1.428524,0.822605,0.808351,0.876106
Elvis Figueroa,1.072374,1.017065,1.206718,1.134739,0.851240,0.514752,0.772422
Tiana Takahashi,0.689661,0.747905,0.999000,0.997307,0.116079,0.665963,0.999000


**Best Case**

In [43]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"unthwackability": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,5.5,2.5,4.5
Elvis Figueroa,4.0,7.0,2.5,5.0
Tiana Takahashi,1.5,5.0,3.0,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,1.145098,1.091270,1.171077,1.428524,0.822605,0.808351,0.876106
Elvis Figueroa,1.433366,1.817065,1.206718,1.134739,0.851240,0.514752,0.772422
Tiana Takahashi,0.992166,1.547905,0.999000,0.997307,0.116079,0.665963,0.999000


### Walk it Off
**A Random Player in your Team's Lineup will have their Moxie Boosted from -20% to +60%.**

Moxie is the player attribute most associated with plate discipline, and increases walks.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [44]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,5.5,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,4.5,4.0,5.0,5.0
Jaxon Buckley,6.5,3.5,3.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Peanut Holloway,3.5,2.5,2.0,3.0
Usurper Violet,3.0,3.0,1.5,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.104763,1.345315,1.152985,0.970888,0.676160,0.283687,0.749895,0.001000,1.025183
Beasley Day,0.882328,1.045160,0.904890,0.612953,0.507377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.875507,0.534025,1.345741,0.959816,1.042462,0.188405,0.708998,0.005052,1.138288
Ruslan Greatness,0.940725,0.622589,1.145689,1.569998,1.270457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.307788,1.567382,1.042101,1.746709,1.730045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.621221,0.812722,0.400621,0.622843,0.741821,0.205246,0.498400,0.001000,0.551015
Peanut Holloway,0.697808,0.656788,0.749014,0.650239,0.555549,0.234531,0.377807,0.207685,0.311857
Usurper Violet,0.568466,0.894648,0.293503,1.022726,0.669795,0.569699,0.283561,0.100000,0.487176


**Best Case**

In [45]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,6.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,3.5,3.5,3.5
Lang Richardson,3.5,0.5,3.5,3.5
Peanut Holloway,3.5,2.5,2.0,3.0
Usurper Violet,3.0,3.0,1.5,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.171387,1.345315,1.152985,0.970888,1.476160,0.283687,0.749895,0.001000,1.025183
Beasley Day,0.947240,1.045160,0.904890,0.612953,1.307377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.913713,0.534025,1.345741,0.959816,1.842462,0.188405,0.708998,0.005052,1.138288
Ruslan Greatness,0.975822,0.622589,1.145689,1.569998,2.070457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.345606,1.567382,1.042101,1.746709,2.530045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.656260,0.812722,0.400621,0.622843,1.541821,0.205246,0.498400,0.001000,0.551015
Peanut Holloway,0.746089,0.656788,0.749014,0.650239,1.355549,0.234531,0.377807,0.207685,0.311857
Usurper Violet,0.602980,0.894648,0.293503,1.022726,1.469795,0.569699,0.283561,0.100000,0.487176


### Feast or Famine
**A Random Player from your Team's Lineup will have their Indulgence Boosted from -20% to +60%.**

Indulgence is the player attribute most associated with baserunners getting advancement on batter outs.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [46]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"indulgence": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_baserunning_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,5.5,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,3.5,3.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Peanut Holloway,3.5,2.5,1.5,3.0
Usurper Violet,3.0,3.0,1.5,4.0


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Eduardo Woodman,0.710447,0.654141,1.077257,0.552728,0.543821,0.844641
Beasley Day,0.696024,0.619583,0.669022,1.189892,0.392646,0.934967
Huber Frumple,0.811571,0.788282,0.737669,0.888626,0.758223,0.819371
Ruslan Greatness,0.999500,1.375092,0.602818,0.693571,0.799000,0.605824
Jaxon Buckley,0.663939,0.689885,1.192203,0.506722,0.313522,0.562351
Lang Richardson,0.675047,0.742498,0.421602,0.845647,0.246945,0.988950
Peanut Holloway,0.343217,0.328968,0.072877,0.279784,0.384838,0.750281
Usurper Violet,0.325767,0.829727,0.996871,0.222526,0.010000,0.015431


**Best Case**

In [47]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"indulgence": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_baserunning_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,5.5,2.0,4.0,3.5
Beasley Day,4.5,3.0,4.0,3.0
Huber Frumple,4.5,2.0,4.5,3.0
Ruslan Greatness,5.0,4.0,5.5,5.0
Jaxon Buckley,6.5,3.5,4.0,3.5
Lang Richardson,3.0,0.5,4.0,3.5
Peanut Holloway,3.5,2.5,2.0,3.0
Usurper Violet,3.0,3.0,2.5,4.0


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Eduardo Woodman,0.777715,0.654141,1.077257,0.552728,1.343821,0.844641
Beasley Day,0.777813,0.619583,0.669022,1.189892,1.192646,0.934967
Huber Frumple,0.872187,0.788282,0.737669,0.888626,1.558223,0.819371
Ruslan Greatness,1.071305,1.375092,0.602818,0.693571,1.599000,0.605824
Jaxon Buckley,0.753652,0.689885,1.192203,0.506722,1.113522,0.562351
Lang Richardson,0.779950,0.742498,0.421602,0.845647,1.046945,0.988950
Peanut Holloway,0.384067,0.328968,0.072877,0.279784,1.184838,0.750281
Usurper Violet,0.491663,0.829727,0.996871,0.222526,0.613191,0.015431


### Revelations
**A Random Player on your Team will have their Omniscience Boosted from -20% to +60%.**

Omniscience is the player attribute most associated with fielding outs and general defense.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [48]:
new_team = []
for player in real_pies.lineup + real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"omniscience": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_defense_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,5.5,2.0,3.5,3.0
Beasley Day,4.5,3.0,3.5,2.5
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,3.5,3.5,3.0
Lang Richardson,3.0,0.5,3.5,3.5
Peanut Holloway,3.5,2.5,2.0,2.5
Usurper Violet,3.0,3.0,1.5,3.5
Henry Marshallow,3.5,4.0,2.5,4.5
Elvis Figueroa,4.0,6.0,2.5,4.5


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Eduardo Woodman,0.559252,0.177478,0.353459,0.834861,0.952460,0.956417
Beasley Day,0.516773,0.155386,0.398364,1.304772,0.457722,0.593574
Huber Frumple,0.552473,0.304837,0.302478,0.490571,1.047842,0.606171
Ruslan Greatness,0.974822,0.970016,0.894059,0.903316,0.848210,1.344688
Jaxon Buckley,0.609951,0.436443,0.470729,0.438125,0.927871,0.415402
Lang Richardson,0.710938,0.843616,0.940524,0.340104,0.390675,0.394329
Peanut Holloway,0.500027,0.148274,0.305188,0.971296,0.777629,0.631750
Usurper Violet,0.748461,0.864183,0.740791,0.533876,0.390291,0.646039
Henry Marshallow,0.885695,0.666120,1.305255,0.761818,0.450244,1.145636
Elvis Figueroa,0.924795,0.913911,1.138358,0.644382,0.683850,0.959366


**Best Case**

In [49]:
new_team = []
for player in real_pies.lineup + real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"omniscience": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_defense_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,5.5,2.0,3.5,4.0
Beasley Day,4.5,3.0,3.5,3.5
Huber Frumple,4.5,2.0,4.0,3.5
Ruslan Greatness,5.0,4.0,5.0,5.5
Jaxon Buckley,6.5,3.5,3.5,4.0
Lang Richardson,3.0,0.5,3.5,4.0
Peanut Holloway,3.5,2.5,2.0,3.5
Usurper Violet,3.0,3.0,1.5,4.5
Henry Marshallow,3.5,4.0,2.5,5.0
Elvis Figueroa,4.0,6.0,2.5,5.0


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Eduardo Woodman,0.786684,0.977478,0.353459,0.834861,0.952460,0.956417
Beasley Day,0.743110,0.955386,0.398364,1.304772,0.457722,0.593574
Huber Frumple,0.714756,1.104837,0.302478,0.490571,1.047842,0.606171
Ruslan Greatness,1.099424,1.770016,0.894059,0.903316,0.848210,1.344688
Jaxon Buckley,0.751181,1.236443,0.470729,0.438125,0.927871,0.415402
Lang Richardson,0.812386,1.643616,0.940524,0.340104,0.390675,0.394329
Peanut Holloway,0.724715,0.948274,0.305188,0.971296,0.777629,0.631750
Usurper Violet,0.853273,1.664183,0.740791,0.533876,0.390291,0.646039
Henry Marshallow,1.037069,1.466120,1.305255,0.761818,0.450244,1.145636
Elvis Figueroa,1.048727,1.713911,1.138358,0.644382,0.683850,0.959366


### Supply Runs
**Each Player in your Team's Shadows with an available Item Slot will open a Generic Sunglasses Crate.**

**Each Player in your Team's Shadows with an available Item Slot will open a Generic Helmet Crate.**


For reference, these items boost the following:


| Item Type  | Attributes | Stat |
| ---------- | ---------- | ---- |
| Sunglasses | +Cinnamon | Vibes |
| Helmet     | +Pressurization | Vibes |


This would provide Generic Sunglasses or Helmets to the following players:

In [50]:
free_items = [x for x in pies.bench + pies.bullpen if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Adrian Melon,2.5,1.0,3.0,2.5
August Obrien,3.5,1.0,3.5,2.0
Mcfarland Vargas,1.5,1.5,4.0,2.0
Twofurious Puddles,2.5,3.5,2.0,3.5
Mummy Cantburn,1.5,2.5,2.0,3.5
Doginic Burgertoes,2.0,2.0,2.0,1.5
Nerd Pacheco,0.5,0.5,0.5,0.5
Yusef Puddles,2.5,2.5,3.0,3.5
Sheev Shriffle,3.5,2.0,2.0,1.0
Adeline Hammer,1.0,1.0,2.5,3.5


### Targeted Evolution
**A random pitcher and a random hitter on your Team will Advance.**

An Advanced player will gain one evolution level, increasing their minimum star level by one for all stats and raising
their stars to the minimum if they are lower than that. It also allows players to hold an additional item.

### Headliners
**Arrange your lineup in order of their Idolatry.**

The exact ordering of our team by Idolatry is unknown. With sufficient team organization this could be used to improve
lineup efficiency.


### Moderation

**Your Team will accept Unruns after each non-loss so that they only non-lose by 1 Run.**

This will reduce our score in games where we Win to the opponent's score plus 1. So if the Pies won a game 7-3, it would
reduce the score to 4-3. This would be helpful for lowering eDensity, as Runs are included in that equation.

### Underhanded

**Make contact, I dare you. Make a random Player in your Team's Rotation Underhanded. Home Runs will be Unruns against them.**

This would cause opponents to earn negative runs when they hit a Home Run against the pitcher with this Modification.

### Subtractor

**A random Player in another Team's Lineup will become a Subtractor.**

The effect of Subtractor is unknown. Interestingly, this would affect another team.

### Go Big? Go Home

**A random Player in another Team's Lineup will become an Underachiever.**

The effect of Underachiever is unknown. Interestingly, this would affect another team.

## Gifts

Gifts are awarded to teams based on contributions from other teams. Each team creates a Wishlist of items that they
would like, and the top items on the wishlist are awarded during the Latesiesta.

### Collector's Editions

Collectors Editions move a Legendary player to the team. Replicas act as copies of that player and will turn to Dust
after elections.

Special Notes:
* Nagomi has fake Attractor stars, this analysis uses the player attributes.
* Chorby is still unstable, and is likely to get incinerated if possible.
* Chorby replicas have 0 soul, so are safe from consumers

In [51]:
york = Player.find_by_name("York Silk")
nagomi = Player.find_by_name("Nagomi Mcdaniel")
nagomi.batting_rating = None
nagomi.baserunning_rating = None
nagomi.defense_rating = None
aldon = Player.find_by_name("Aldon Cashmoney")
goodwin = Player.find_by_name("Goodwin Morin")

pm = Player.find_by_name("Pitching Machine")
chorby = Player.find_by_name("Chorby Soul")

display(Markdown("#### Batting"))
display(set_heatmap(get_batting_stlats([york, nagomi, aldon, goodwin] + pies.lineup), maxVal=1.5))

display(Markdown("Adding York Silk"))
display(pandas.DataFrame(move_player(pies, york, position='lineup')))

display(Markdown("Adding Nagomi Mcdaniel"))
display(pandas.DataFrame(move_player(pies, nagomi, position='lineup')))

display(Markdown("Adding Aldon Cashmoney"))
display(pandas.DataFrame(move_player(pies, aldon, position='lineup')))

display(Markdown("Adding Goodwin Morin"))
display(pandas.DataFrame(move_player(pies, goodwin, position='lineup')))

display(Markdown("#### Pitching"))
display(set_heatmap(get_pitching_stlats([pm, chorby] + pies.rotation), maxVal=1.5))

display(Markdown("Adding Pitching Machine"))
display(pandas.DataFrame(move_player(pies, pm, position='rotation')))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='rotation')))

#### Batting

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
York Silk,0.856037,1.086786,0.774307,0.949095,0.784552,0.596532,0.286926,0.227032,0.926663
Nagomi Mcdaniel,0.702909,0.576162,0.771588,0.801071,0.922936,0.367576,0.564414,0.692533,0.388730
Aldon Cashmoney,0.998344,1.058616,1.070545,0.723468,1.110157,0.342420,0.728757,0.158430,0.415462
Goodwin Morin,1.116151,1.011205,1.054999,1.804185,1.724130,0.001000,1.146978,0.054253,1.040647
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Beasley Day,0.904594,1.045160,0.904890,0.612953,0.707377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.887107,0.534025,1.345741,0.959816,1.242462,0.188405,0.708998,0.005052,1.138288
Ruslan Greatness,0.951096,0.622589,1.145689,1.569998,1.470457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.318562,1.567382,1.042101,1.746709,1.930045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.632443,0.812722,0.400621,0.622843,0.941821,0.205246,0.498400,0.001000,0.551015


Adding York Silk

,Pies
batting_change,-1.0039%
pitching_change,+0.0000%
baserunning_change,-5.5802%
defense_change,-3.5463%


Adding Nagomi Mcdaniel

...

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Pitching Machine,1.420057,1.479285,1.364489,1.504418,0.775101,0.812889,0.953238
Chorby Soul,0.000299,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Elvis Figueroa,1.173083,1.217065,1.206718,1.134739,0.851240,0.514752,0.772422
Tiana Takahashi,0.776416,0.947905,0.999000,0.997307,0.116079,0.665963,0.999000


Adding Pitching Machine

,Pies
batting_change,+0.0000%
pitching_change,+12.8530%
baserunning_change,+0.0000%
defense_change,+0.0000%


Adding Chorby Soul

,Pies
batting_change,+0.0000%
pitching_change,-22.6409%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Solo Editions

These gifts provide a random player with a benefit for the lateseason and postseason.

* Fourth Strike: Player will be out on 4 strikes instead of 3.
* Walk in the Park: Player will earn a walk on 2 balls rather than 3.
* Ambitious: Player will overperform in the postseason.

### Team Editions

These gifts provide a team-wide benefit for the lateseason and postseason.

* Life of the Party: Increases the improvement of each party while in Party Time.
* Late to the Party: Team will overperform in the Lateseason.
* Home Field Advantage: Team will start each home game with 1 run. Playoff games alternate between home and away each game.
* Fireproof: Team will be safe from incinerations. Pies only have 1 Solar Eclipse game in the lateseason.
* Soundproof: Team will be safe from Feedback. Pies have 2 Feedback weather games in the lateseason.
* Gravity: Team will be safe from Reverb. Pies only have 1 Reverb weather game in the lateseason.

### Handcrafted Drops

These will provide Items which have additional stat increases and random special effects.

|Type | Attribute | Stat |
| --- | --- | --- |
| Cannon | +Overpowerment | Pitching |
| Jersey | +Musclitude | Batting |
| Necklace | +Moxie | Batting |
| Phone | ??? | ??? |
| Quill | ??? | ??? |
| Ring | +Ruthlessness | Pitching |
| Sunglasses | +Cinnamon | Vibes |

See below for a list of players that can receive them.

**Jersey, Necklace**

In [52]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

,Batting,Pitching,Baserunning,Defense
Peanut Holloway,6.0,4.0,4.0,5.5


**Cannon, Quill, Ring**

In [53]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

,Batting,Pitching,Baserunning,Defense
Elvis Figueroa,4.0,6.0,2.5,5.0


**Phone, Sunglasses**

In [54]:
free_items = [x for x in pies.lineup + pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Peanut Holloway,6.0,4.0,4.0,5.5
Elvis Figueroa,4.0,6.0,2.5,5.0


### Soul Patches

**Back by Popular Demand! Give 1 Soul to your least Soul Full player. Repeat 10 times.**

See below for final Soul totals. Low soul is dangerous, as Players will become Redacted if their soul hits 0 and
Consumers will drain 1 soul per chomp.

In [55]:
players = deepcopy(pies.rotation) + deepcopy(pies.lineup)
for i in range(0, 10):
    players.sort(key=lambda x: x.soul)
    worst = players[0]
    worst.soul += 1
    players = [x for x in players if x.id != worst.id] + [worst]
players.sort(key=lambda x: x.soul)
x = pandas.DataFrame()
for p in players:
    old = [x for x in pies.rotation + pies.lineup if x.id == p.id][0]
    x = x.append(pandas.Series({"Soul Added": p.soul - old.soul,"Resulting Soul": p.soul}, name=p.name))
x

,Resulting Soul,Soul Added
Eduardo Woodman,5.0,1.0
Ruslan Greatness,5.0,1.0
Usurper Violet,5.0,1.0
Peanut Holloway,5.0,2.0
Lang Richardson,6.0,0.0
Henry Marshallow,6.0,1.0
Elvis Figueroa,6.0,2.0
Tiana Takahashi,6.0,2.0
Beasley Day,7.0,0.0
Huber Frumple,7.0,0.0


### Bargain Bin

**Give a random Player on your Team the Best Item from the Bargain Bin!**

The Bargain Bin will retrieve one random previously dropped item. Below is a list of all non-base items not currently
held by players.

In [56]:
all_items = Item.load_all()
rare_items = [x for x in all_items.values() if len(x.adjustments) > 1]
players = Player.load_all()
carried_items = []
for p in players.values():
    carried_items.extend(p.items)
dropped_items = [x for x in rare_items if x not in carried_items]
dropped_items.sort(key=lambda x: len(x.adjustments), reverse=True)

table = []
for i in dropped_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)

,Durability,Modifications
Weird Plant-Based Bat,3,[]
Cool Holey Shoes,3,[]
Sharp Jersey of Charisma,3,[]
Sharp Glove of Wisdom,3,[]
Rubber Skate Bat of Strength,4,[Pro Skater]
Aluminum Blood Sunglasses,3,[]
Sharp Leg Bat,3,[]
Weird Shoes,2,[]
Weird Bat,2,[]
Weird Cannon,2,[]


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
